In [1]:
import numpy as np

In [3]:
# removing bad frame

# loading full data set
foldername    = 'ch2/01_Photometry/Fixed_Exact/Run11/'      # location of data file
file_full = 'ch2_datacube_full_AORs464.dat'                 # name of data file

path = foldername + filename_full

# Flux, Flux Uncertainty, Time, x-centroid, y-centroid, x-PSF width, y-PSF width, bg flux, bg flux err
flux   = np.loadtxt(path, usecols=[0], skiprows=1)
ferr   = np.loadtxt(path, usecols=[1], skiprows=1)
time   = np.loadtxt(path, usecols=[2], skiprows=1)
xo     = np.loadtxt(path, usecols=[3], skiprows=1)
yo     = np.loadtxt(path, usecols=[4], skiprows=1)
xw     = np.loadtxt(path, usecols=[5], skiprows=1)
yw     = np.loadtxt(path, usecols=[6], skiprows=1)
bgflux = np.loadtxt(path, usecols=[7], skiprows=1)
bgferr = np.loadtxt(path, usecols=[8], skiprows=1)

In [11]:
ind_del = np.arange(0, flux.size, 64)   # np.arange(frame id, size of array, every 64 frames)

# deleting frames
flux   = np.delete(flux, ind_del)
ferr   = np.delete(ferr, ind_del)
time   = np.delete(time, ind_del)
xo     = np.delete(xo  , ind_del)
yo     = np.delete(yo  , ind_del)
xw     = np.delete(xw  , ind_del)
yw     = np.delete(yw  , ind_del)
bgflux = np.delete(bgflux, ind_del)
bgferr = np.delete(bgferr, ind_del)

bin_size = 63

In [13]:
def binning_data(data, size):
    '''
    Median bin an array.

    Parameters
    ----------
    data     : 1D array
        Array of data to be binned.

    size     : int
        Size of bins.

    Returns
    -------
    binned_data: 1D array
        Array of binned data.

    binned_data: 1D array
        Array of standard deviation for each entry in binned_data.
    '''
    data = np.ma.masked_invalid(data) 
    reshaped_data   = data.reshape((int(len(data)/size), size))
    binned_data     = np.ma.median(reshaped_data, axis=1)
    binned_data_std = np.std(reshaped_data, axis=1)
    return binned_data, binned_data_std

In [14]:
# removing bad frame
binned_flux, binned_flux_std = binning_data(np.asarray(flux), bin_size)
binned_time, binned_time_std = binning_data(np.asarray(time), bin_size)
binned_xo, binned_xo_std     = binning_data(np.asarray(xo), bin_size)
binned_yo, binned_yo_std     = binning_data(np.asarray(yo), bin_size)
binned_xw, binned_xw_std     = binning_data(np.asarray(xw), bin_size)
binned_yw, binned_yw_std     = binning_data(np.asarray(yw), bin_size)
binned_bg, binned_bg_std     = binning_data(np.asarray(bgflux), bin_size)
binned_bg_err, binned_bg_err_std = binning_data(np.asarray(bgferr), bin_size)

In [16]:
save_full = 'new_ch2_datacube_full_AORs464.dat'                 # name of data file
save_bin  = 'new_ch2_datacube_binned_AORs464.dat'                 # name of data file

FULL_data = np.c_[flux, ferr, time, xo, yo, xw, yw, bgflux, bgferr]
FULL_head = 'Flux, Flux Uncertainty, Time, x-centroid, y-centroid, x-PSF width, y-PSF width, bg flux, bg flux err'
BINN_data = np.c_[binned_flux, binned_flux_std, binned_time, binned_time_std, binned_xo, binned_xo_std, binned_yo,
                          binned_yo_std, binned_xw, binned_xw_std, binned_yw, binned_yw_std, binned_bg, binned_bg_std, 
                          binned_bg_err, binned_bg_err_std]
BINN_head = 'Flux, Flux std, Time, Time std, x-centroid, x-centroid std, y-centroid, y-centroid std, \
        x-PSF width, x-PSF width std, y-PSF width, y-PSF width std'
pathFULL  = foldername + save_full
pathBINN  = foldername + save_bin
np.savetxt(pathFULL, FULL_data, header = FULL_head)
np.savetxt(pathBINN, BINN_data, header = BINN_head)